In [3]:
import os,sys,inspect
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from datetime import datetime
from agents.Base import BaseAgent, run
from agents.BlueAgents import *
from data.preprocessing import data_split, get_price, DataProcessor
import pandas as pd
import numpy as np
from agents.wrappers.ObservationWrapper import NormalizedEnv
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from env.EnvStock_train import StockEnvTrain
from env.EnvStock_trade import StockEnvTrade
from env.EnvStock_val import StockEnvValidation
from env.crypto.SingleCryptoEnv import SingleCryptoEnv
from stable_baselines3 import A2C, PPO
from env.crypto.SingleCryptoEnv import SingleCryptoEnv
from env.BaseEnv import EnvConfig

%matplotlib inline
%load_ext autoreload
%autoreload 2
"""df = pd.read_csv('../datasets/downew30.csv')
indata = data_split(df, '2015-01-01','2015-05-05')
target_step =len(indata.Date.unique())
target_step
train_env = StockEnvTrain(indata, indata,stock_dim=30, config=EnvConfig)
agent_builder = DRLAgent(StockEnvTrain, indata, EnvConfig)
model_kwargs = {
    'learning_rate':0.002,
    'batch_size':8,
    'gamma':0.95,
    'seed':42,
    'net_dimension':160,
    'target_step':160,
    'eval_time_gap':10
}
model = agent_builder.get_model('ppo', model_kwargs)
agent_builder.train_model(model, './',total_timesteps=100)"""

"df = pd.read_csv('../datasets/downew30.csv')\nindata = data_split(df, '2015-01-01','2015-05-05')\ntarget_step =len(indata.Date.unique())\ntarget_step\ntrain_env = StockEnvTrain(indata, indata,stock_dim=30, config=EnvConfig)\nagent_builder = DRLAgent(StockEnvTrain, indata, EnvConfig)\nmodel_kwargs = {\n    'learning_rate':0.002,\n    'batch_size':8,\n    'gamma':0.95,\n    'seed':42,\n    'net_dimension':160,\n    'target_step':160,\n    'eval_time_gap':10\n}\nmodel = agent_builder.get_model('ppo', model_kwargs)\nagent_builder.train_model(model, './',total_timesteps=100)"

In [1]:
import tensorboard

In [2]:
getattr(tensorboard, '__version__')

'2.8.0'

In [2]:
from env.BaseEnv import EnvConfig


In [10]:
EnvConfig.__dict__.items()

dict_items([('__module__', 'env.BaseEnv'), ('INITIAL_ACCOUNT_BALANCE', 1000000), ('HMAX_NORMALIZE', 100), ('REWARD_SCALING', 1e-05), ('TRANSACTION_FEE_PERCENT', 0.001), ('SHORT_FEE', 5e-05), ('INDICATORS', ['macd', 'tema', 'boll', 'boll_ub', 'boll_lb', 'close_10_sma_xd_close_50_sma', 'close_30_sma', 'close_60_sma', 'wr_10', 'rsi_30', 'rsi_14', 'cci_30', 'dx_30', 'vr', 'atr', 'dma', 'volume_delta', 'volume']), ('OBSERVATIONS', 20), ('INDEX_OBSERVATIONS', 3), ('REWARD_INTERVAL', 3), ('seed', 42), ('use_turbulance', False), ('__dict__', <attribute '__dict__' of 'EnvConfig' objects>), ('__weakref__', <attribute '__weakref__' of 'EnvConfig' objects>), ('__doc__', None)])

In [17]:
', '.join([str(k + ':' + str(v) + ' | ') for k,v in EnvConfig.__dict__.items() if '__' not in k])

"INITIAL_ACCOUNT_BALANCE:1000000 | , HMAX_NORMALIZE:100 | , REWARD_SCALING:1e-05 | , TRANSACTION_FEE_PERCENT:0.001 | , SHORT_FEE:5e-05 | , INDICATORS:['macd', 'tema', 'boll', 'boll_ub', 'boll_lb', 'close_10_sma_xd_close_50_sma', 'close_30_sma', 'close_60_sma', 'wr_10', 'rsi_30', 'rsi_14', 'cci_30', 'dx_30', 'vr', 'atr', 'dma', 'volume_delta', 'volume'] | , OBSERVATIONS:20 | , INDEX_OBSERVATIONS:3 | , REWARD_INTERVAL:3 | , seed:42 | , use_turbulance:False | "

In [4]:
df = pd.read_csv('../datasets/downew30.csv')
cwd = '../AgentResult'

In [5]:
df.ticker.unique()

array(['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GS', 'HD',
       'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT',
       'NKE', 'PFE', 'PG', 'RTX', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT',
       'XOM'], dtype=object)

In [3]:
indata = data_split(df, '2010-01-01','2021-12-25')
indtwo = data_split(df, '2015-01-01','2015-01-03')

In [4]:
S = len(indata) // 4
N = int(len(indata)/S)
frames = [ indata.iloc[i*S:(i+1)*S].copy() for i in range(N+1) ]

In [5]:
target_step =len(indata.Date.unique())
target_step

2329

In [6]:
#dloader = DataProcessor('crypto',['btc'],'2019-01-01','2022-01-01')

In [28]:
#dataframe = dloader.get_crypto_price('btc_usdt_1d.csv', 'binance', 3, 'BTC/USDT', '1d', '2018-03-0200:00:00Z', 1000)

In [29]:
#btc = pd.read_csv('./data/raw/Binance/btc_usdt_1m.csv',names=['Date','open','high','low','close','volume'])

In [30]:
indtwo.Date = pd.to_datetime(indtwo.Date)

In [32]:
train_env = VecNormalize(DummyVecEnv([lambda: StockEnvTrain(indata, indata,stock_dim=30, config=EnvConfig)]))
train_envtwo = VecNormalize(DummyVecEnv([lambda: StockEnvTrade(df=indtwo, index_df=indtwo,
                                                               unique_trade_date=[datetime.now()],
                                                               initial=False,
                                                               stock_dim=30, config=EnvConfig, 
                                                               previous_state=state_trade)]),)

NameError: name 'state_trade' is not defined

In [10]:
a = [-1,1, 1]

In [11]:
sorted(a)

[-1, 1, 1]

In [43]:
env = StockEnvTrain(indata, indata,stock_dim=30, config=EnvConfig)
train_env = NormalizedEnv(env)


In [12]:
s = train_env.reset()
t = train_envtwo.reset()

NameError: name 'train_envtwo' is not defined

In [49]:
env = StockEnvTrain(indata, indata,stock_dim=30, config=EnvConfig)
train_env = NormalizedEnv(env)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [101]:
trades = pd.read_csv('results/trade_memory_ppo.csv')

# Own agent

1. Winner params
'''
model_kwargs = {
    'learning_rate':0.0027,
    'batch_size':64,
    'gamma':0.99,
    'seed':42069,
    'net_dimension':256, # Change this dimension to be more dynamic
    'target_step':5000,
    'eval_time_gap':10
}
model_name = 'jim-ppo-v2'
model_type = 'ppo'
'''

In [20]:
model_kwargs = {
    'learning_rate':0.0027,
    'batch_size':64,
    'gamma':0.99,
    'seed':42069,
    'net_dimension':256, # Change this dimension to be more dynamic
    'target_step':90000,
    'eval_time_gap':10
}
model_name = 'jim-ppo-90000'
model_type = 'ppo'

In [21]:
agent_builder = DRLAgent(model_name,model_type, StockEnvTrain, 
                         StockEnvValidation, 
                         StockEnvTrade, indata, EnvConfig)


In [22]:
#model = agent_builder.get_model('ppo', model_kwargs)

- [ ] Change the timesteps version make sure it does reiterate with same agent
- [x] Normalize Inputs for the model
- [x] Normalize reward function
- [ ] Vectorized paralel envs
- [x] start trading from 2016
- [ ] Add PBT to evaluator
- [x] Change neet dimension problem here you cannot make it another than 160 
- [ ] Create a database for models

In [ ]:
agent_builder.run_prediction(model_kwargs, total_timesteps=1000)

======Model training from:  2010-01-01 to  2016-01-04
======Training Agents with the population of 1========
| Remove cwd: ./trained_models
Using device:  cpu
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.
0  8.90e+04    7.21 |    7.21    0.0     62     0 |   -0.49   0.79   0.37  -0.50
| UsedTime:     783 | SavedDir: ./trained_models | Total Step: 89032
======jim-ppo-90000 Validation from:  2016-01-04 to  2016-04-04
======Recurrent PPO Validation from:  2016-01-04 to  2016-04-05
Total reward at validation for Reccurent PPO 0
Best params,  {'learning_rate': 0.002, 'batch_size': 8, 'gamma': 0.95, 'seed': 42, 'net_dimension': 160, 'target_step': 160, 'eval_time_gap': 10}
======Trading from:  2016-04-05 to  2016-07-05
Using device:  cpu
previous_total_asset:1000000
end_total_asset:1020342.9017355616
total_asset_change:20342.90173556155
Total cash is: 0.5668547327392837$ and total hol

In [20]:
200 / 1.25

160.0

In [11]:
round(0.43,1)

0.4